Now, let me show the steps to follow to avoid finding a "lucky"
strategy or an overfitted strategy like in our previous example:
1. Find the data, create new features, and split the dataset.
2. Create a strategy and OPTIMIZE it on the train set
3. Backtest the strategy on the test set: keep it if it is good or stop
here. Do not change the strategy's parameters; the strategy is not
profitable. It does not matter; we will try another one!
We need to understand that the more we touch our test set to adapt the
strategy, the more we will have bad performances in the future (in live
trading).

The second rule to follow when we backtest a
strategy is not to consider the big profit.

Always adapt the analysis of your backtest to your
target strategy

In [1]:
def find_timestamp_extremum (data , df_lowest_timeframe): 
    """ 
    :params: data(highest timeframe OHLCV data), df_lowest_timeframe (lowest timeframe OHLCV data) 
    :return: data with three new columns: Low_time (TimeStamp), High_time (TimeStamp), High_first (Boolean) 
    """
    
    # Set new columns
    data[ "Low_time" ] = np.nan 
    data[ "High_time" ] = np.nan 
    data[ "First" ] = np.nan
    
    # Loop to find out which of the Take Profit and Stop loss appears first
    for i in tqdm( range ( len (data) - 1 )):
        
        # Extract values from the lowest timeframe dataframe
        start = data.iloc[i:i + 1 ].index[ 0 ] 
        end = data.iloc[i + 1 :i + 2 ].index[ 0 ] 
        row_lowest_timeframe = df_lowest_timeframe.loc[start:end]. iloc[: -1 ]
        
        # Extract Timestamp of the max and min over the period (highest timeframe)
        try : 
            high = row_lowest_timeframe[ "high" ].idxmax() 
            low = row_lowest_timeframe[ "low" ].idxmin() 
            data.loc[start, "Low_time" ] = low 
            data.loc[start, "High_time" ] = high 
        except Exception as e: 
            print (e) 
            data.loc[start, "Low_time" ] = start 
            data.loc[start, "High_time" ] = start
            
    # Find out which appears first
    data.loc[data[ "High_time" ] > data[ "Low_time" ], "First" ] = 1 
    data.loc[data[ "High_time" ] < data[ "Low_time" ], "First" ] = 2 
    data.loc[data[ "High_time" ] == data[ "Low_time" ], "First" ] = 0
        
    # Verify the number of row without both TP and SL on same time
    percentage_garbage_row= len (data.loc[data[ "First" ]== 0 ].dropna()) / len (data) * 100 

    #if percentage_garbage_row<95: 
    print ( f "WARNINGS: Garbage row: { '%.2f' % percentage_garbage_row} %" )

    # Transform the columns in datetime columns
    data.High_time = pd.to_datetime(data.High_time) 
    data.Low_time = pd.to_datetime(data.Low_time)

    # We delete the last row because we can't find the extremum
    data = data.iloc[: -1 ]

    # Specific to the current data
    if "timestamp" is data.columns: 
        del data[ "timestamp" ] 
            
    return data

It will be very simple. To compute the returns using a Take-profit /
Stop-loss exit strategy, we need to consider four cases:
1. We open a buy position, and we touch the TP first (High price)
2. We open a buy position, and we touch the SL first (Low price)
3. We open a sell position, and we touch the TP first (Low price)
4. We open a sell position, and we touch the SL first (High price)

In [ ]:
# Create random signals
np.random.seed( 70 ) 
values = [ -1 , 0 , 1 ] 
df[ "Signal" ] = [np.random.choice(values , p=[ 0.10 , 0.80 , 0.10 ]) for _ in range ( len (df))]

In [ ]:
def run_tp_sl ( data , leverage = 1 , tp = 0.015 , sl = -0.015 , cost = 0.00 ): 
    """ 
        :params (mandatory): data(have to contain a High_time and a Low_time columns) 
        :params (optional): leverage=1, tp=0.015, sl=-0.015, cost=0.00 
        :return: data with three new columns: Low_time (TimeStamp), High_time (TimeStamp), High_first (Boolean) 
    """

    # Set some parameters
    buy= False sell= False data[ "duration" ] = 0 
    
    for i in range ( len (data)):

        # Extract data
        row = data.iloc[i] 

        ######## OPEN BUY ######## 
        if buy== False and row[ "Signal" ]== 1 : 
            buy = True 
            open_buy_price = row[ "open" ] 
            open_buy_date = row.name 
            
        #VERIF 
        if buy: 
            var_buy_high = (row[ "high" ] - open_buy_price) / open_buy_price 
            var_buy_low = (row[ "low" ] - open_buy_price) / open_buy_price

        # VERIF FOR TP AND SL ON THE SAME CANDLE 
        if (var_buy_high > tp) and (var_buy_low < sl):

            # IF TP / SL ON THE SAME TIMESTAMP, WE DELETE THE TRADE RETURN 
            if row[ "Low_time" ] == row[ "High_time" ]: 
                pass 
            elif row[ "First" ]== 2 : 
                data.loc[row.name, "returns" ] = (tp-cost) * leverage 
                data.loc[row.name, "duration" ] = row.High_time – open_buy_date 
            elif row[ "First" ]== 1 : 
                data.loc[row.name, "returns" ] = (sl-cost) * leverage 
                data.loc[row.name, "duration" ] = row.Low_time – open_buy_date 
                
                buy = False 
                open_buy_price = None 
                var_buy_high = 0 
                var_buy_low = 0 
                open_buy_date = None 
                
            elif var_buy_high > tp: 
                data.loc[row.name, "returns" ] = (tp-cost) * leverage 
                buy = False 
                open_buy_price = None 
                var_buy_high = 0 
                var_buy_low = 0 
                data.loc[row.name, "duration" ] = row.High_time – open_buy_date 
                open_buy_date = None 
                
            elif var_buy_low < sl: 
                data.loc[row.name, "returns" ] = (sl-cost) * leverage 
                buy = False 
                open_buy_price = None 
                var_buy_high = 0 
                var_buy_low = 0 
                data.loc[row.name, "duration" ] = row.Low_time – open_buy_date 
                open_buy_date = None 
                
            ######## OPEN SELL ######## 
            if sell== False and row[ "Signal" ]== -1 : 
                sell = True 
                open_sell_price = row[ "open" ] 
                open_sell_date = row.name 
            # VERIF 
            if sell: 
                var_sell_high = -(row[ "high" ] - open_sell_price) / open_sell_price 
                var_sell_low = -(row[ "low" ] - open_sell_price) / open_sell_price 
                
                if (var_sell_low > tp) and (var_sell_high < sl): 
                    if row[ "Low_time" ] == row[ "High_time" ]: 
                        pass 
                    elif row[ "First" ]== 1 : #À INVERSER POUR LE BUY 
                        data.loc[row.name, "returns" ] = (tp-cost) * leverage 
                        data.loc[row.name, "duration" ] = row.Low_time – open_sell_date 
                    elif row[ "First" ]== 2 : 
                        data.loc[row.name, "returns" ] = (sl-cost) * leverage 
                        data.loc[row.name, "duration" ] = row.High_time – open_sell_date 
                        sell = False 
                        open_sell_price = None 
                        var_sell_high = 0 
                        var_sell_low = 0 
                        open_sell_date = None 
                    
                    elif var_sell_low > tp: 
                        data.loc[row.name, "returns" ] = (tp-cost) * leverage 
                        sell = False 
                        open_sell_price = None 
                        var_sell_high = 0 
                        var_sell_low = 0 
                        data.loc[row.name, "duration" ] = row.Low_time – open_sell_date 
                        open_sell_date = None 
                        
                    elif var_sell_high < sl: 
                        data.loc[row.name, "returns" ] = (sl-cost) * leverage 
                        sell = False 
                        open_sell_price = None 
                        var_sell_high = 0 
                        var_sell_low = 0 
                        data.loc[row.name, "duration" ] = row.High_time – open_sell_date 
                        open_sell_date = None

    # Put 0 when we have missing values
    data[ "returns" ] = data[ "returns" ].fillna(value= 0 )
    
    return data

def
monte_carlo
(
data
,
method
=
"simple"
):
1
random_returns = []
data[
"returns"
] = data[
"returns"
].fillna(value=
0
)
for
_
in
tqdm(
range
(
100
)):
returns = data[
"returns"
]
-10
**
-100
2
np.random.shuffle(returns)
random_returns.append(returns)
if
method==
"simple"
:
df_ret =
pd.DataFrame(random_returns).transpose().cumsum()*
100
cur_ret = data[
"returns"
].cumsum()*
100
else
:
df_ret = ((
1
+pd.DataFrame(random_returns).transpose()).cumprod()
-1
)*
100
cur_ret = ((
1
+data[
"returns"
]).cumprod()
-1
)*
100
p_90 = np.percentile(df_ret,
99
, axis=
1
)
p_50 = np.percentile(df_ret,
50
, axis=
1
)
p_10 = np.percentile(df_ret,
1
, axis=
1
)
plt.figure(figsize=(
20
,
8
))
plt.plot(df_ret.index, p_90, color=
"#39B3C7"
)
plt.plot(df_ret.index, p_50, color=
"#39B3C7"
)
plt.plot(df_ret.index, p_10, color=
"#39B3C7"
)
plt.plot(cur_ret, color=
"blue"
, alpha=
0.60
, linewidth=
3
,
label=
"Current returns"
)
plt.fill_between(df_ret.index, p_90, p_10,
p_90>p_10, color=
"#669FEE"
, alpha=
0.20
,
label=
"Monte carlo area"
)
plt.ylabel(
"Cumulative returns %"
, size=
13
)
plt.title(
"MONTE CARLO SIMULATION"
, size=
20
)
plt.legend()
plt.show()

In [ ]:
# Bonus
def profitable_month_return ( p ): 
    total = 0 positif = 0 
    
    r=[]

    # Loop on each different year
    for year in p.index.strftime( "%y" ).unique(): 
        e = [] 
        nbm = p.loc[p.index.strftime( "%y" )==year].index.strftime( "%m" ).unique()

        # Loop on each different month
        for mois in nbm: 
            monthly_values = p.loc[p.index.strftime( "%y:%m" )== f " {year} : {mois} " ] 
            sum_ = monthly_values.sum()

        # Verifying that there is at least 75% of the values
        if len(monthly_values) > 15 :

            # Computing sum return
            s = monthly_values.sum() 
            
            if s > 0: 
                positif += 1 
                
            else: 
                pass 
            
            total += 1 
            
        else: 
            pass 
        
        e.append(sum_) 
        r.append(e) 
        r[0] = [ 0 for _ in range ( 12 - len (r[ 0 ]))] + r[ 0 ] r[ -1 ]= r[ -1 ] + [ 0 for _ in range ( 12 - len (r[ -1 ]))] 
        
        return pd.DataFrame(r, columns=[ 
            "January" , 
            "February" , 
            "March" , 
            "April" , 
            "May" , 
            "June" , 
            "July" , 
            "August" , 
            "September" , 
            "October" , 
            "November" , 
            "December" 
        ], index=p.index.strftime( "%y" ).unique()) 
    
    def heatmap(data): 
        htm = profitable_month_return(data[ "returns" ])* 100 
        htm.index.name = "Year" 
        htm.index = [ f "20 {idx} " for idx in htm.index] 
        
        plt.figure(figsize=( 20 , 8 )) 
        pal = sns.color_palette( "RdYlGn" ,n_colors= 15 ) 
        sns.heatmap(htm, annot= True , cmap =pal, vmin= -100 , vmax= 100 ) 
        
        plt.title("Heatmap Monthly returns") 
        plt.show()

1. Time underwater
: the percentage of time we have a drawdown
below 0. It allows us to understand the percentage of time we will
earn money. It is essential to understand that the time does not
give the intensity of the loss: we can have a 1% time underwater
and a max drawdown of 100%, so we will lose all of our capital.
This metric should always be combined to the maximum
drawdown.
2. Trade lifetime
: the average time of a trade position. The more
scalping-like the strategy, the more the trade lifetime is essential.
3. Assets under management
: It is the dollar value of our portfolio
at each time (it is a vector). We usually compute the average AUM
(assets under management).
4. Long – short ratio
: Number of long positions over the number of
sell positions. So, if we work with a long-short strategy, the more
the value is close to 0.5, the better it is.
5. Number of trades
: very important to understand how many
trades you take and see if it is accorded to your trading plan: a
strategy with 12 trades over a year is not a significative backtest
but 500 is.
6. Annualized returns
: It is essential to compute the annualized
return to be able to compare several strategies between them.
7. Correlation to underlying:
Pearson’s correlation is usually
between the strategy returns and the underlying returns. The more
the value is close to 1, the more the strategy long the asset and the
more the correlation is close to -1, the more the strategy short the
asset. So, the strategy is not adding a lot of value if the correlation
is close to -1 or 1.
8. HIT ratio:
percentage of winning trade. It must be associated
with the risk-reward ratio.
9. Risk-reward ratio:
Essential to understand how many our risk is
compared to our targeted reward. Again, HIT and risk-reward
ratios (R ratios) are the two faces of the same coin
10. Sharpe ratio:
An essential financial metric, it allows us to
understand the benefits of return of one more percentage risk. It
must be annualized.
11. Sortino ratio:
Same metric as the Sharpe ratio, but we compute
the risk using the downward volatility instead of the classic
volatility for the Sharpe ratio.
12. Beta:
It will give us some indications about how much the
strategy is correlated to the market (SP500, for example) (more
explanation in chapter 5)
13. Alpha:
tells us how the strategy overperforms or underperforms
the market. (More explanation in chapter 5)
14. Information ratio:
it will allow us to compare the risk-reward
couple of the strategy to the benchmark risk-reward couple.
15. Risk-free asset return:
Consider the risk-free asset returns over
the period (annualized) to compare them to the annualized
strategy returns.
16. VaR:
Will give us the worst loss we can make with a 5% error
threshold. We can compute it on the period that you want: the
worst loss per day, month or year for example. (More explanation
in chapter 5)
17. cVaR:
Like the VaR but some difference in the computation.
(More explanation in chapter 5)


Use 20 backtest metrics will not render your strategy
better. In my opinion, 5 good indicators are clearly
enough to have a quick overview

So far, we have analyzed the performance of our strategy on only one
possible path. Indeed, the past is only one possible path between an
infinity.
To analyze different possible paths, we can use Monte Carlo
simulations. To create one Monte-Carlo simulation, we will randomly
take the strategy returns and reorganize the data.

In [ ]:
def monte_carlo ( data , method = "simple" ):
    random_returns = [] 
    data[ "returns" ] = data[ "returns" ].fillna(value= 0 ) 
    
    for _ in tqdm( range ( 100 )): 
        returns = data[ "returns" ] -10 * -100
        np.random.shuffle(returns) 
        random_returns.append(returns) 
        
    if method== "simple" : 
        df_ret = pd.DataFrame(random_returns).transpose().cumsum() * 100 
        cur_ret = data[ "returns" ].cumsum() * 100 
    else : 
        df_ret = (( 1 +pd.DataFrame(random_returns).transpose()).cumprod() -1 ) * 100 
        cur_ret = (( 1 +data[ "returns" ]).cumprod() -1 )* 100 
        
    p_90 = np.percentile(df_ret, 99 , axis= 1 ) 
    p_50 = np.percentile(df_ret, 50 , axis= 1 ) 
    p_10 = np.percentile(df_ret, 1 , axis= 1 ) 
    plt.figure(figsize=( 20 , 8 )) 
    plt.plot(df_ret.index, p_90, color= "#39B3C7" ) 
    plt.plot(df_ret.index, p_50, color= "#39B3C7" ) 
    plt.plot(df_ret.index, p_10, color= "#39B3C7" ) 
    plt.plot(cur_ret, color= "blue" , alpha= 0.60 , linewidth= 3 , label= "Current returns" ) 
    plt.fill_between(df_ret.index, p_90, p_10, p_90 > p_10, color= "#669FEE" , alpha= 0.20 , label= "Monte carlo area" ) 
    plt.ylabel( "Cumulative returns %" , size= 13 ) 
    plt.title( "MONTE CARLO SIMULATION" , size= 20 )
    plt.legend() 
    plt.show()

def
run_tsl
(
data
,
leverage
=
1
,
tp
=
0.015
,
sl
=
-0.015
,
tsl
=
0.001
,
cost
=
0.00
):
"""
:params (mandatory): data(have to contain a High_time and a Low_time
columns)
:params (optional): leverage=1, tp=0.015, sl=-0.015, cost=0.00
:return: data with three new columns: Low_time (TimeStamp),
High_time (TimeStamp), High_first (Boolean)
"""
tpl = tp – tsl
1

In [ ]:
def run_tsl(data, leverage=1, tp=0.015, sl=-0.015, tsl=0.001, cost=0.00): 
    """ 
        :params (mandatory): data(have to contain a High_time and a Low_time columns) 
        :params (optional): leverage=1, tp=0.015, sl=-0.015, cost=0.00 :return: data with three new columns: Low_time (TimeStamp), High_time (TimeStamp), High_first (Boolean) 
    """ 
    tpl = tp – tsl